## Imports
inkl. Bildimport

In [19]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [20]:
# pip install transformers

In [21]:
# pip install timm # for DetrConvEncoder

In [22]:
# pip install pytorch-lightning

In [31]:
# pip install pycocotools

     ---------------------------------------- 84.9/84.9 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd
import numpy as np
import torch, torchvision, torchaudio

from transformers import AutoImageProcessor, DetrForObjectDetection
from PIL import Image, ImageDraw
import sys
import requests

In [24]:
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
#url = "http://images.cocodataset.org/val2017/000000039769.jpg"
#image = Image.open(requests.get(url, stream=True).raw)

local = "C:/Users/Michael/Desktop/old/BAA/Annotierte_Bilder/2024-02-21-Schadenserkennung-Annotationen/images/LB00000510.png"
image = Image.open(local).convert("RGB")

image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.8, target_sizes=target_sizes)[
    0
]

# print(results)
boxes = []
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    boxes.append(box)
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )
print(boxes)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Detected book with confidence 0.819 at location [845.3, 386.9, 1102.25, 463.73]
Detected book with confidence 0.831 at location [847.19, 298.4, 1093.29, 359.54]
Detected book with confidence 0.85 at location [846.5, 815.68, 1097.37, 875.0]
Detected book with confidence 0.855 at location [1092.68, 307.47, 1335.5, 385.39]
Detected book with confidence 0.919 at location [573.83, 563.34, 850.96, 677.42]
Detected book with confidence 0.83 at location [1092.95, 370.19, 1334.24, 444.72]
Detected book with confidence 0.822 at location [846.94, 350.84, 1100.97, 429.94]
Detected book with confidence 0.86 at location [846.28, 745.25, 1090.86, 811.76]
Detected book with confidence 0.815 at location [595.95, 814.88, 832.39, 884.79]
Detected book with confidence 0.871 at location [579.68, 466.76, 852.87, 545.79]
Detected book with confidence 0.82 at location [845.75, 531.48, 1092.19, 613.67]
Detected book with confidence 0.881 at location [842.38, 678.51, 1094.85, 763.49]
[[845.3, 386.9, 1102.25, 46

In [26]:
polys = [[854.47, 300.44, 920.71, 613.65], [512.43, 296.58, 606.01, 614.75], [958.31, 319.3, 1027.05, 621.06], [893.3, 300.9, 982.65, 616.84]]

with Image.open("LB00000510.png") as im:

    draw = ImageDraw.Draw(im)
    # y = [(1121.1928567300467,278.23551354730927),(1121.1928567300467 + 52.5394965665439,278.23551354730927 + 288.9672311159914)]
    # draw.rectangle(y, -1, 128, 10)
    for x in boxes:
        print(x)
        draw.rectangle(x, -1, 128, 10)
    im.show()
    #im.save(sys.stdout, "PNG")

[845.3, 386.9, 1102.25, 463.73]
[847.19, 298.4, 1093.29, 359.54]
[846.5, 815.68, 1097.37, 875.0]
[1092.68, 307.47, 1335.5, 385.39]
[573.83, 563.34, 850.96, 677.42]
[1092.95, 370.19, 1334.24, 444.72]
[846.94, 350.84, 1100.97, 429.94]
[846.28, 745.25, 1090.86, 811.76]
[595.95, 814.88, 832.39, 884.79]
[579.68, 466.76, 852.87, 545.79]
[845.75, 531.48, 1092.19, 613.67]
[842.38, 678.51, 1094.85, 763.49]


# Training Loop
Inspired by: https://github.com/NielsRogge/Transformers-Tutorials/tree/master/DETR

In [28]:
from transformers import DetrConfig, DetrImageProcessor, DetrForObjectDetection
import torchvision
import os

In [33]:
class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, processor, train=True):
        ann_file = os.path.join(img_folder, "coco_train.json" if train else "coco_val.json")
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self.processor = processor

    def __getitem__(self, idx):
        # read in PIL image and target in COCO format
        # feel free to add data augmentation here before passing them to the next step
        img, target = super(CocoDetection, self).__getitem__(idx)

        # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.processor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        target = encoding["labels"][0] # remove batch dimension

        return pixel_values, target

In [35]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")


# Create Train and Validation Dataset
# TODO im dqa das kleinere Datenset in train und val aufteilen, vielleicht auch ein testset extrahieren?
train_dataset = CocoDetection(img_folder='../../../old/BAA/Annotierte_Bilder/2024-02-21-Schadenserkennung-Annotationen', processor=processor)
val_dataset = CocoDetection(img_folder='../../../old/BAA/Annotierte_Bilder/2024-02-21-Schadenserkennung-Annotationen', processor=processor, train=False)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../../../old/BAA/Annotierte_Bilder/2024-02-21-Schadenserkennung-Annotationen\\coco_train.json'

In [40]:
import os
os.path.exists('../../../old/BAA/Annotierte_Bilder/2024-02-21-Schadenserkennung-Annotationen/')


True